# Imports

for PyArabic
https://github.com/linuxscout/pyarabic/blob/master/doc/features.md

In [16]:
!pip install pyarabic
import nltk
nltk.download('punkt')
import re
import pyarabic.araby as araby
from pyarabic.araby import is_arabicrange, strip_diacritics
import gensim
from gensim.models import Word2Vec
import tensorflow as tf
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy
from numpy import array
from sklearn.preprocessing import OneHotEncoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Inputs


In [5]:
 def read_input(file_path):
    with open(file_path, 'r') as file:
        content =  file.readlines()
    return content

In [6]:
train_dataset = read_input('train.txt')
val_dataset = read_input('val.txt')

#Preprocessing

In [7]:
def remove_diacritics_and_save(text):
    stripped = araby.strip_diacritics(text)
    diacritics_dic = [{i: char} for i,char in enumerate(text) if char not in stripped]
    return stripped, diacritics_dic

In [60]:
def remove_non_arabic_char(text):
    regex = re.compile('[^؀-ۿ ,.،؛]')
    result = regex.sub('', text)
    return result

In [9]:
def splitting(text):
    return text.split()

In [108]:
def segmenter(text):
    pattern = r"[.؟!؛،,]+|,"
    sentences = re.split(pattern, text)
    # Remove empty strings from the list
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

In [109]:
segmented = segmenter(train_dataset[5])
segmented

for s in segmented:
    print(word_tokenize(s))

['وَقَوْلُهُ', '(', 'لَزِمَتْهُ', 'لِمَا', 'قُلْنَا', ')', 'يُرِيدُ', 'قَوْلَهُ']
['لِأَنَّ', 'الْجَمْعَ', 'بَيْنَهُمَا', 'مَشْرُوعٌ', 'فِي', 'حَقِّ', 'الْآفَاقِيِّ']


In [67]:
arabic_only = remove_non_arabic_char(train_dataset[2])
print('without non arabic char: ', arabic_only)
splitted = splitting(arabic_only)
print(splitted)

stripped_words = []
dict_of_diacritics = []
for s in splitted:
    stripped, dict_diacritics = remove_diacritics_and_save(s)
    segmented = segmenter(stripped)
    stripped_words.append(segmented)
    dict_of_diacritics.append(dict_diacritics)
print("stripped words = ", stripped_words)
print("dict of diacritics = ", dict_of_diacritics)

without non arabic char:   قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ  أَيْ الْوَصِيَّةُ  قَوْلُهُ مَا مَرَّ  أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا لَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي أَنَّهَا لَا تَبْطُلُ ، وَعِبَارَةُ الْكَنْزِ وَلَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي لَمْ يَتَعَيَّنْ غَنَمُهُ إنْ كَانَتْ انْتَهَتْ ا ه سم  قَوْلُهُ فَيُعْطَى وَاحِدَةً مِنْهَا إلَخْ  كَمَا لَوْ كَانَتْ مَوْجُودَةً عِنْدَ الْوَصِيَّةِ وَالْمَوْتِ ، وَلَا يَجُوزُ أَنْ يُعْطَى وَاحِدَةً مِنْ غَيْرِ غَنَمِهِ فِي الصُّورَتَيْنِ وَإِنْ تَرَاضَيَا ؛ لِأَنَّهُ صُلْحٌ عَلَى مَجْهُولٍ مُغْنِي وَنِهَايَةٌ قَالَ ع ش قَوْلُهُ وَاحِدَةً مِنْهَا أَيْ كَامِلَةً ، وَلَا يَجُوزُ أَنْ يُعْطَى نِصْفَيْنِ مِنْ شَاتَيْنِ ؛ لِأَنَّهُ لَا يُسَمَّى شَاةً وَقَوْلُهُ وَلَا يَجُوزُ أَنْ يُعْطَى وَاحِدَةً مِنْ غَيْرِ غَنَ

#Feature Extraction

In [13]:
arabic_alphabet = [
    "أ", "ب", "ت", "ث", "ج", "ح", "خ", "د", "ذ", "ر", "ز", "س", "ش", "ص",
    "ض", "ط", "ظ", "ع", "غ", "ف", "ق", "ك", "ل", "م", "ن", "ه", "و", "ي"
]
encoder = OneHotEncoder(sparse=False, categories=[arabic_alphabet])
one_hot_encoded = encoder.fit_transform([[category] for category in arabic_alphabet])

print(one_hot_encoded.shape)

(28, 28)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [132]:
sentences = []
# sentences = [train_dataset[0]] + [train_dataset[1]]
sentences_diactrics_dict = []
for text in train_dataset:
    sentence = remove_non_arabic_char(text)
    sentence, d = remove_diacritics_and_save(sentence)
    segmented = segmenter(sentence)
    sentences_diactrics_dict.append(d)
    for s in segmented:
        tokens = word_tokenize(s)
        sentences.append(tokens)
    print(len(sentence))
    print(segmented)


Streaming output truncated to the last 5000 lines.
67
['كما لو انفرد به من أصله وله عمل رحى عليه ونحوه  و  الماء  المشترك']
113
['لأنه قد وصل إلى ولي الجناية حقه', 'فخلص الحق في الأخذ للمولى', 'وله رأي في إيصال المنفعة لنفسه', 'والامتناع من ذلك']
542
['أشبهت القن لمفهوم قوله صلى الله عليه وسلم  هي معتقة عن دبر منه  وقوله  فهي معتقة من بعده  فدل على أنها قبل ذلك باقية في الرق  إلا في تدبير  فلا يصح تدبيرها لأنه لا فائدة فيه إذ الاستيلاد أقوى منه حتى أنه لو طرأ عليه أبطله كما تقدم  أو ما ينقل الملك كبيع  فلا يصح بيع أم الولد  غير كتابة  فتصح كتابتها ويقدم  وكهبة ووصية ووقف  لحديث ابن عمر مرفوعا  نهى عن بيع أمهات الأولاد وقال  لا يبعن ولا يوهبن ولا يورثن', 'يستمتع بها السيد ما دام حيا', 'فإذا مات فهي حرة  رواه الدارقطني ورواه مالك في الموطأ والدارقطني من طريق آخر عن ابن عمر عن عمر موقوفا']
268
['فيصلي  الإمام بمن حضره ركعتين كالعيد وتقدم  ثم يخطب خطبة واحدة  على المنبر والناس جلوس عنده', 'لأنه لم ينقل غيره عنه صلى الله عليه وسلم  يفتتحها  أي الخطبة  بالتكبير  تسعا نسقا  كخطبة العيد  لقول 

In [134]:
print(sentences[0])

['قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزركشي']


In [164]:
longest_length = max(len(lst) for lst in sentences)
print(longest_length)
index_of_longest_list = max(range(len(sentences)), key=lambda i: len(sentences[i]))
print(index_of_longest_list)
print(sentences[index_of_longest_list])

307
139034
['ش', 'قوله', 'إن', 'عمر', 'بن', 'الخطاب', 'استشار', 'في', 'الخمر', 'يشربها', 'الرجل', 'وجواب', 'علي', 'يدل', 'على', 'أنه', 'إنما', 'استشار', 'في', 'قدر', 'الحد', 'وإنما', 'كان', 'ذلك', 'لأن', 'الأصح', 'أنه', 'لم', 'يتقرر', 'في', 'زمن', 'النبي', 'صلى', 'الله', 'عليه', 'وسلم', 'بمعنى', 'أنه', 'لم', 'يحد', 'فيه', 'حدا', 'بقول', 'يعلم', 'لا', 'يزاد', 'عليه', 'ولا', 'ينقص', 'عنه', 'وإنما', 'كان', 'يضرب', 'مقدارا', 'قدرته', 'الصحابة', 'واختلفوا', 'في', 'تقديره', 'يدل', 'على', 'ذلك', 'ما', 'روي', 'عن', 'علي', 'بن', 'أبي', 'طالب', 'رضي', 'الله', 'عنه', 'أنه', 'قال', 'ما', 'من', 'رجل', 'أقمت', 'عليه', 'حدا', 'فمات', 'فأجد', 'في', 'نفسي', 'منه', 'شيئا', 'إلا', 'شارب', 'الخمر', 'فإنه', 'إن', 'مات', 'فيه', 'وديته', 'لأن', 'رسول', 'الله', 'صلى', 'الله', 'عليه', 'وسلم', 'لم', 'يبينه', 'ومعنى', 'ذلك', 'أنه', 'لم', 'يحده', 'بقول', 'يحصره', 'ويمنع', 'الزيادة', 'فيه', 'والنقص', 'منه', 'فحدوه', 'باجتهادهم', 'وروى', 'أنس', 'أتي', 'النبي', 'صلى', 'الله', 'عليه', 'وسلم', 'برجل', 'قد', 'شرب', 'ال

In [172]:
# Create Word2Vec model (using Skip-gram, you can also use CBOW)
model = Word2Vec(sentences=sentences, vector_size=300, window=20, sg=1, min_count=1)
model.train(sentences, total_examples=len(sentences), epochs=10)
# Save the model (optional)
model.save("word2vec_model")

# Load the model (optional)
# model = Word2Vec.load("word2vec_model")

# Get the vector representation of a word
word_vector = model.wv["قال"]

word = "أو"
# Find similar words
similar_words = model.wv.most_similar(word, topn=5)

# Print the results
print(f"Vector representation of {word}:", word_vector)
print(f"Similar words to {word}:", similar_words)

Vector representation of أو: [-6.99151084e-02  3.05549949e-01 -1.71290725e-01 -1.68057159e-02
  1.93623349e-01 -2.39820838e-01  2.10489348e-01  4.52250868e-01
  1.24369346e-01 -6.09374642e-02 -2.86353249e-02 -4.67340433e-04
  2.16338217e-01  3.36695672e-03 -5.73583357e-02 -9.78622679e-03
  1.45897582e-01  2.13593945e-01 -2.97793075e-02  1.07618831e-01
 -2.45435476e-01  3.40700485e-02 -4.71212715e-02 -1.23616494e-01
  1.47061974e-01 -1.41845271e-01 -2.65673310e-01  6.04879782e-02
 -1.70172029e-03 -2.09539101e-01 -2.46850178e-01 -3.65248993e-02
  2.08183825e-01 -1.12353303e-01 -6.57649264e-02  1.42010152e-01
 -1.03371799e-01 -1.09847613e-01 -4.94952016e-02 -1.02198392e-01
 -7.38092586e-02 -1.11796044e-01 -1.54698536e-01  1.62741728e-02
 -3.30022983e-02  1.01303622e-01 -3.92164886e-02  1.62024185e-01
  2.93581307e-01  1.92659333e-01  1.52386665e-01  2.95715183e-02
  6.80400282e-02 -8.55665058e-02 -2.23495029e-02  2.51394585e-02
  4.59075458e-02  3.02601587e-02 -9.24976449e-03 -2.09428534e

In [171]:
word_vector = model.wv["قال"]

word = "جميل"
# Find similar words
similar_words = model.wv.most_similar(word, topn=5)

# Print the results
print(f"Vector representation of {word}:", word_vector)
print(f"Similar words to {word}:", similar_words)

AttributeError: ignored

In [170]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample dataset
arabic_text = ["مرحبا", "اليوم", "في", "مدرسة"]
diacritics = ["مَرْحَبًا", "الْيَوْمِ", "فِي", "مَدْرَسَةِ"]

# Tokenize characters and diacritics
tokenizer_chars = Tokenizer(char_level=True)
tokenizer_chars.fit_on_texts(arabic_text)

tokenizer_diacritics = Tokenizer(char_level=True)
tokenizer_diacritics.fit_on_texts(diacritics)

# Convert characters to sequences
X_sequences = tokenizer_chars.texts_to_sequences(arabic_text)
y_sequences = tokenizer_diacritics.texts_to_sequences(diacritics)

# Padding sequences to have the same length
X_padded = pad_sequences(X_sequences, padding='post')
y_padded = pad_sequences(y_sequences, padding='post')

# Define the model
model_RNN = Sequential([
    Embedding(input_dim=len(tokenizer_chars.word_index) + 1, output_dim=32, input_length=X_padded.shape[1]),
    LSTM(64, return_sequences=True),
    TimeDistributed(Dense(len(tokenizer_diacritics.word_index) + 1, activation='softmax'))
])

# Compile the model
model_RNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_RNN.fit(X_padded, y_padded, epochs=10, batch_size=1, validation_split=0.2)

# Example predictions
sample_text = ["السلام"]
sample_sequence = tokenizer_chars.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequence, padding='post', maxlen=X_padded.shape[1])
predictions = model_RNN.predict(sample_padded)

# Decode predictions
predicted_diacritics = tokenizer_diacritics.sequences_to_texts(np.argmax(predictions, axis=-1))

print(f"Input Text: {sample_text[0]}")
print(f"Predicted Diacritics: {predicted_diacritics[0]}")

Epoch 1/10


ValueError: ignored